In [1]:
import sys
sys.path
sys.path.append('/notebooks/Sophus/py')

from sophus import *
import numpy as np
from sympy import *

In [2]:
## xyz quaternion ==> se(3)
def normalize(ww,wx,wy,wz):# make first number positive
    q = [ww, wx, wy, wz]
    ## Find first negative
    idx = -1
    for i in range(len(q)):
        if q[i] < 0:
            idx = i
            break
        elif q[i] > 0:
            break
    # -1 if should not filp, >=0  flipping index
    if idx >= 0:
        ww = ww * -1
        wx = wx * -1
        wy = wy * -1
        wz = wz * -1
    return ww, wx, wy, wz  
    

def xyzQuaternion2se3(x,y,z,ww,wx,wy,wz):
    trans = Matrix([x,y,z])
    ww, wx, wy, wz = normalize(ww, wx, wy, wz)
        
    q_real = ww
    q_img = Matrix([wx, wy, wz])
    q = Quaternion(q_real,q_img)
    R = So3(q)
    
    RT = Se3(R, trans)
    
    return RT.log()

In [3]:
ww, wx, wy, wz = normalize(-0.764,0.002,0.612,-0.204)
print(ww, wx, wy, wz)


print("se3 in R^6")
RT_se3_R6 = xyzQuaternion2se3(9,10,13,ww, wx, wy, wz)
pprint(RT_se3_R6)

print("se3 in R^4x4")
hat = Se3.hat(RT_se3_R6)
pprint(hat)

print("se3 in R^6")
vee = Se3.vee(hat)
pprint(vee)

0.764 -0.002 -0.612 0.204
se3 in R^6
⎡  18.3424761127696  ⎤
⎢                    ⎥
⎢  7.14975327850079  ⎥
⎢                    ⎥
⎢  4.54085273856874  ⎥
⎢                    ⎥
⎢-0.00434794589099048⎥
⎢                    ⎥
⎢ -1.33047144264309  ⎥
⎢                    ⎥
⎣ 0.443490480881028  ⎦
se3 in R^4x4
⎡        0           -0.443490480881028    -1.33047144264309   18.342476112769
⎢                                                                             
⎢0.443490480881028           0            0.00434794589099048  7.1497532785007
⎢                                                                             
⎢1.33047144264309   -0.00434794589099048           0           4.5408527385687
⎢                                                                             
⎣        0                   0                     0                  0       

6⎤
 ⎥
9⎥
 ⎥
4⎥
 ⎥
 ⎦
se3 in R^6
⎡  18.3424761127696  ⎤
⎢                    ⎥
⎢  7.14975327850079  ⎥
⎢                    ⎥
⎢  4.54085273856874

In [35]:
print("SE3")
M_SE3 = Se3.exp(RT_se3_R6)#-0.764,0.002,0.612,-0.204)
M_SE3 = M_SE3.matrix()
pprint(M_SE3)

print("acc SE3")
M_SE3_acc = M_SE3 * M_SE3
pprint(M_SE3_acc)

SE3
⎡0.167563458884244  -0.309307303022423  -0.936083051627228  9.0 ⎤
⎢                                                               ⎥
⎢0.314203988558398  0.916748344768267   -0.246674534434821  10.0⎥
⎢                                                               ⎥
⎢0.934450823115236  -0.252787390234633  0.250799111875662   13.0⎥
⎢                                                               ⎥
⎣        0                  0                   0            1  ⎦
acc SE3
⎡-0.94383165364392  -0.0987555679594194  -0.315323876953199   -4.75408157142 ⎤
⎢                                                                            ⎥
⎢0.110189871804384   0.805598151132593   -0.582125253741393  18.7885503970556⎥
⎢                                                                            ⎥
⎢0.311512442338211  -0.584173738444959   -0.749467171782543  22.1425719600744⎥
⎢                                                                            ⎥
⎣        0                   0                   0  

In [36]:
from pyquaternion import Quaternion

In [78]:
## SE3 to x y z quaternion
def SE3toXYZQuaternion(matrix):
    trans = np.array([matrix[0,3], matrix[1,3], matrix[2,3]]).astype(float)

    q = np.array(matrix[0:3,0:3]).astype(float)
    
    q8d = Quaternion(matrix=q)
    r = np.array([q8d.real]).astype(float)
    v = np.array(q8d.imaginary).astype(float)
    
    q_con = np.concatenate([trans, r, v])
    
    return q_con

xq = SE3toXYZQuaternion(M_SE3)
print(xq)

[ 9.00000000e+00  1.00000000e+01  1.30000000e+01  7.64053486e-01
 -2.00014001e-03 -6.12042844e-01  2.04014281e-01]
